# Get Conservation Scores

Get PhyloP conservation scores for ensembl transcripts

In [1]:
import pandas as pd
import requests
import sys
from datasets import dataset_list
from tqdm import tqdm

In [2]:

def get_conservation(chr, start, end, genome):
    api_url = 'http://api.genome.ucsc.edu/getData/track'
    if genome == 'hg38':
        track = 'phyloP100way'
    elif genome == 'mm39':
        track = 'phyloP35way'
    else:
        raise ValueError('Genome not recognized')
    chrom = 'chr' + chr
    params = {
        'genome': genome,
        'track': track,
        'start': start,
        'end': end,
        'chrom': chrom
    }
    results = requests.get(api_url, data=params)
    if results.ok:
        value_df = (pd.DataFrame([pd.Series(x) for x in pd.read_json(results.content.decode('utf8'))[chrom].values])
                    .rename(columns={'value': 'conservation'}))
    else:
        raise ValueError(results.reason)
    return value_df


def build_request_url(ext, server="https://rest.ensembl.org"):
    request_url = "/".join([server, ext])
    return request_url


def handle_results(results):
    if not results.ok:
        results.raise_for_status()
        sys.exit()
    decoded = results.json()
    return decoded


def get_transcript_info(transcript):
    base_transcript = transcript.split('.')[0]
    request_url = build_request_url("/lookup/id/" + base_transcript + "?expand=1")
    r = requests.get(request_url, headers={"Content-Type": "application/json"})
    decoded = handle_results(r)
    exon_df = pd.DataFrame(decoded['Exon'])
    trans_sr = pd.Series(decoded['Translation'])
    chr = decoded['seq_region_name']
    return exon_df, trans_sr, chr


def get_exon_conservation(exon_df, chr, genome):
    conservation_dict = {}
    for i, row in exon_df.set_index('id').iterrows():
        # subtract one since the nucleotide conservation corresponds to the "end" index
        conservation_dict[i] = get_conservation(chr, row['start'] - 1, row['end'], genome)
        # get the conservation of i
    conservation_df = (pd.concat(conservation_dict)
                       .reset_index(level=0)
                       .reset_index(drop=True)
                       .rename({'level_0': 'exon_id',
                                'end': 'genomic position'}, axis=1)
                       .drop('start', axis=1))
    return conservation_df


def get_transcript_conservation(transcript_id, target_strand, genome):
    exon_df, trans_sr, chr = get_transcript_info(transcript_id)
    # only include translated positions
    exon_df['start'] = exon_df['start'].apply(lambda x: max(x, trans_sr['start']))
    exon_df['end'] = exon_df['end'].apply(lambda x: min(x, trans_sr['end']))
    exon_df = exon_df[exon_df['end'] > exon_df['start']].reset_index(drop=True)
    conservation_df = get_exon_conservation(exon_df, chr, genome)
    conservation_df['Transcript Base'] = transcript_id
    if target_strand == '-':
        ascending = False
    else:
        ascending = True
    conservation_df = (conservation_df
                       .sort_values('genomic position', ascending=ascending)
                       .reset_index(drop=True))
    conservation_df['target position'] = conservation_df.index + 1
    conservation_df['chromosome'] = chr
    conservation_df['genome'] = genome
    return conservation_df


In [3]:
data_list = list()
for ds in dataset_list:
    if ds.endogenous:
        data_list.append(ds)

design_list = list()
for ds in tqdm(data_list):
    ds.load_data()
    ds.set_sgrnas()
    design_list.append(ds.get_designs())

100%|██████████| 8/8 [00:23<00:00,  2.99s/it]


In [4]:
design_df = (pd.concat(design_list)
             .drop_duplicates())
transcript_refseq_df = (design_df[['Target Transcript', 'Strand of Target', 'Target Total Length']]
                        .drop_duplicates())
transcript_refseq_df['Transcript Base'] = (transcript_refseq_df['Target Transcript']
    .str.split('.', expand=True)[0])
transcript_refseq_df['genome'] = transcript_refseq_df['Transcript Base'].apply(lambda trans:
                                                                               'mm39' if 'MUS' in trans else 'hg38')
len(transcript_refseq_df)


2998

In [5]:
transcript_conservation_list = []
failed_list = []
for i, row in tqdm(transcript_refseq_df.iterrows(), total=transcript_refseq_df.shape[0]):
    try:
        transcript_conservation_list.append(get_transcript_conservation(row['Transcript Base'],
                                                                        row['Strand of Target'],
                                                                        row['genome']))
    except:
        print(row)
        failed_list.append(row)

100%|██████████| 2998/2998 [3:51:43<00:00,  4.64s/it]  


Target Transcript      ENST00000611665.4
Strand of Target                       +
Target Total Length                  714
Transcript Base          ENST00000611665
genome                              hg38
Name: 712, dtype: object
Target Transcript      ENST00000622530.4
Strand of Target                       +
Target Total Length                 2391
Transcript Base          ENST00000622530
genome                              hg38
Name: 3429, dtype: object
Target Transcript      ENST00000368563.6
Strand of Target                       -
Target Total Length                 2709
Transcript Base          ENST00000368563
genome                              hg38
Name: 3857, dtype: object
Target Transcript      ENST00000377815.5
Strand of Target                       +
Target Total Length                 1860
Transcript Base          ENST00000377815
genome                              hg38
Name: 4381, dtype: object
Target Transcript      ENST00000618014.1
Strand of Target                    

In [8]:
print('Failed transcripts: ' + str(len(failed_list)))

Failed transcripts: 12


In [10]:
transcript_conservation_df = (pd.concat(transcript_conservation_list))
conservation_size = (transcript_conservation_df.groupby('Transcript Base')
                     .agg(nrows = ('genome', 'count'))
                     .reset_index()
                     .merge(transcript_refseq_df))
print('Number of transcripts missing conservation scores: ' + str((conservation_size['nrows'] != conservation_size['Target Total Length']).sum()))

Number of transcripts missing conservation scores: 22


In [12]:
missing_conservation = conservation_size[conservation_size['nrows'] != conservation_size['Target Total Length']]
missing_conservation

,Transcript Base,nrows,Target Transcript,Strand of Target,Target Total Length,genome
14,ENSMUST00000001479,2630,ENSMUST00000001479.4,-,2631,mm39
497,ENSMUST00000030769,1302,ENSMUST00000030769.5,+,1428,mm39
743,ENSMUST00000043914,1977,ENSMUST00000043914.6,+,2064,mm39
985,ENSMUST00000076313,2900,ENSMUST00000076313.12,-,2862,mm39
1043,ENSMUST00000087114,804,ENSMUST00000087114.3,-,810,mm39
1160,ENSMUST00000106054,1587,ENSMUST00000106054.2,+,1695,mm39
1169,ENSMUST00000107498,6705,ENSMUST00000107498.8,+,6729,mm39
1513,ENST00000222800,921,ENST00000222800.7,-,948,hg38
1725,ENST00000263083,1317,ENST00000263083.10,+,1332,hg38
1828,ENST00000276072,4914,ENST00000276072.7,+,5682,hg38


In [16]:
out_conservation = transcript_conservation_df[~(transcript_conservation_df['Transcript Base']
                                                .isin(missing_conservation['Transcript Base']))].reset_index(drop=True)
out_conservation['Transcript Base'].nunique()

2964

In [18]:
out_conservation.to_parquet('../data/interim/conservation.parquet.gzip',
                            compression='gzip')